In [3]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 100 #1000

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        states, targets_f = [], []
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target 
            # Filtering out states and targets for training
            states.append(state[0])
            targets_f.append(target_f[0])
        history = self.model.fit(np.array(states), np.array(targets_f), epochs=1, verbose=0)
        # Keeping track of loss
        loss = history.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [6]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(50): #500
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                loss = agent.replay(batch_size)
                # Logging training loss every 10 timesteps
                if time % 10 == 0:
                    print("episode: {}/{}, time: {}, loss: {:.4f}"
                        .format(e, EPISODES, time, loss))  
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0/100, score: 20, e: 1.0
episode: 1/100, score: 11, e: 1.0
episode: 2/100, time: 0, loss: 3.9826
episode: 2/100, score: 9, e: 0.96
episode: 3/100, time: 0, loss: 5.2799
episode: 3/100, time: 10, loss: 2.2725
episode: 3/100, time: 20, loss: 5.0726
episode: 3/100, time: 30, loss: 0.7435
episode: 3/100, score: 40, e: 0.78
episode: 4/100, time: 0, loss: 0.7268
episode: 4/100, time: 10, loss: 4.2620
episode: 4/100, score: 12, e: 0.74
episode: 5/100, time: 0, loss: 2.1536
episode: 5/100, time: 10, loss: 0.8248
episode: 5/100, score: 16, e: 0.68
episode: 6/100, time: 0, loss: 5.4935
episode: 6/100, time: 10, loss: 4.4028
episode: 6/100, score: 16, e: 0.63
episode: 7/100, time: 0, loss: 2.3717
episode: 7/100, time: 10, loss: 2.3142
episode: 7/100, score: 11, e: 0.59
episode: 8/100, time: 0, loss: 3.7450
episode: 8/100, time: 10, loss: 4.3191
episode: 8/100, score: 14, e: 0.55
episode: 9/

episode: 54/100, time: 40, loss: 0.2965
episode: 55/100, time: 0, loss: 0.2844
episode: 55/100, time: 10, loss: 1.1166
episode: 55/100, time: 20, loss: 3.4319
episode: 55/100, time: 30, loss: 3.0668
episode: 55/100, time: 40, loss: 4.1276
episode: 55/100, score: 43, e: 0.01
episode: 56/100, time: 0, loss: 9.8362
episode: 56/100, time: 10, loss: 0.3260
episode: 56/100, time: 20, loss: 8.6723
episode: 56/100, time: 30, loss: 6.5309
episode: 56/100, time: 40, loss: 2.4521
episode: 57/100, time: 0, loss: 0.3689
episode: 57/100, time: 10, loss: 5.3890
episode: 57/100, time: 20, loss: 0.6988
episode: 57/100, time: 30, loss: 0.6523
episode: 57/100, time: 40, loss: 0.4979
episode: 58/100, time: 0, loss: 0.2539
episode: 58/100, time: 10, loss: 0.3310
episode: 58/100, time: 20, loss: 3.7237
episode: 58/100, time: 30, loss: 0.2571
episode: 58/100, time: 40, loss: 4.2576
episode: 59/100, time: 0, loss: 4.7339
episode: 59/100, time: 10, loss: 0.2348
episode: 59/100, time: 20, loss: 1.9266
episode: 

episode: 95/100, time: 40, loss: 0.0056
episode: 96/100, time: 0, loss: 0.0070
episode: 96/100, time: 10, loss: 0.0111
episode: 96/100, time: 20, loss: 0.0165
episode: 96/100, time: 30, loss: 0.0068
episode: 96/100, time: 40, loss: 0.0143
episode: 97/100, time: 0, loss: 0.0055
episode: 97/100, time: 10, loss: 0.0097
episode: 97/100, time: 20, loss: 0.0071
episode: 97/100, time: 30, loss: 0.0065
episode: 97/100, time: 40, loss: 0.0105
episode: 98/100, time: 0, loss: 0.0112
episode: 98/100, time: 10, loss: 0.0075
episode: 98/100, time: 20, loss: 0.0069
episode: 98/100, time: 30, loss: 0.0059
episode: 98/100, time: 40, loss: 0.0057
episode: 99/100, time: 0, loss: 0.0099
episode: 99/100, time: 10, loss: 0.0060
episode: 99/100, time: 20, loss: 0.0041
episode: 99/100, time: 30, loss: 0.0052
episode: 99/100, time: 40, loss: 0.0061
